In [0]:
# 10 merge into sample 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("AppendOnlyDemo") \
    .enableHiveSupport() \
    .getOrCreate()

# 1) Create an initial “Silver” table with one row per user
initial = [(1, "Alice", 100), (2, "Bob", 150)]
cols    = ["user_id", "name", "balance"]
df_init = spark.createDataFrame(initial, cols)

# Write it out as Delta
silver_path = "/mnt/data/silver_demo"
spark.sql("DROP TABLE IF EXISTS demo.silver_no_merge")
df_init.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)
spark.sql(f"""
  CREATE TABLE demo.silver_no_merge
  USING DELTA
  LOCATION '{silver_path}'
""")

# 2) Simulate an update in Bronze: Alice’s balance → 200
update = [(1, "Alice", 200)]
df_upd  = spark.createDataFrame(update, cols)

# **Append** it into the Silver folder, without MERGE
df_upd.write.format("delta") \
    .mode("append") \
    .save(silver_path)

# 3) Query the “Silver” table
spark.sql("""
  SELECT * 
    FROM demo.silver_no_merge
   WHERE user_id = 1
""").show()


In [0]:
from delta.tables import DeltaTable

# Assume spark session and demo.silver_demo path from above

# 1) Register a fresh Silver table for merge
silver_path2 = "/mnt/data/silver_demo_merge"
spark.sql("DROP TABLE IF EXISTS demo.silver_with_merge")
df_init.write.format("delta") \
    .mode("overwrite") \
    .save(silver_path2)
spark.sql(f"""
  CREATE TABLE demo.silver_with_merge
  USING DELTA
  LOCATION '{silver_path2}'
""")

# 2) Use DeltaTable API to MERGE the same update
silver = DeltaTable.forName(spark, "demo.silver_with_merge")

silver.alias("t") \
    .merge(
        df_upd.alias("s"),               # source = the one updated row
        "t.user_id = s.user_id"          # match on the business key
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

# 3) Query the merged table
spark.sql("""
  SELECT * 
    FROM demo.silver_with_merge
   WHERE user_id = 1
""").show()
